In [ ]:
import matplotlib.pyplot as plt
import GridWorld
import numpy as np
import pandas as pd

In [ ]:
robot=GridWorld.grid_world()

In [ ]:
robot.initial_state()

In [ ]:
robot.s

In [ ]:
def print_values(V, rows,columns):
    for i in range(rows):
        print("---------------------------")
        for j in range(columns):
            v = V.get((i,j), 0)
            if v >= 0:
                print(" %.2f|" % v, end="")
            else:
                print("%.2f|" % v, end="") # -ve sign takes up an extra space
        print("")
def print_policy(policy,rows,columns):
    for i in range(rows):
        print("---------------------------")
        for j in range(columns):
              a = policy.get((i,j), ' ')
              print("  %s  |" % a, end="")
        print("")

In [ ]:
def play_episode(policy,max_steps=20):
    s=robot.initial_state()
    a = np.random.choice(robot.ACTION_SPACE) # first action is uniformly random
    # keep track of all states and rewards encountered
    episodestates = [s]
    episodeactions = []
    episoderewards = [0]
    steps = 0
    done=False
    while not done:
        a = policy[s]
        episodeactions.append(a)
        next_s, r, done=robot.step(a)
        # update states and rewards lists
        episodestates.append(next_s)
        episoderewards.append(r)
        s=next_s
        steps += 1
        if steps >= max_steps:
            break
    # update state
    # note: there is no need to store the final terminal state
    s = next_s
    # we want to return:
    # gamestates  = [s(0), s(1), ..., S(T)]
    # gamerewards = [R(0), R(1), ..., R(T)]
    # gameactions = [a(0), a(1), ...,a(T-1) ]
    return episodestates, episodeactions,episoderewards

In [ ]:
policy = {}
for s in robot.actions.keys():
    policy[s] = np.random.choice(robot.actions.get(s))
policy

In [ ]:
play_episode(policy,max_steps=20)

In [ ]:
def max_dict(d):
    # returns the argmax (key) and max (value) from a dictionary
    # put this into a function since we are using it so often

    # find max val
    max_val = max(d.values())

    # find keys corresponding to max val
    max_keys = [key for key, val in d.items() if val == max_val]

    return np.random.choice(max_keys), max_val

In [ ]:
# initialize Q(s,a) and returns
Q = {}
sample_counts = {}
state_sample_count={}
for s in robot.States:
    if s in robot.actions.keys(): # not a terminal state
        Q[s] = {}
        sample_counts[s] = {}
        state_sample_count[s] = 0
        for a in robot.ACTION_SPACE:
            Q[s][a] = 0
            sample_counts[s][a] = 0

In [ ]:
Q

In [ ]:
sample_counts

In [ ]:
GAMMA=0.9
# repeat until convergence
deltas = []
for it in range(200000):
    if it % 1000 == 0:
        print(it)

    # generate an episode using pi
    biggest_change = 0
    episodestates, episodeactions, episoderewards = play_episode(policy)
    # create a list of only state-action pairs for lookup
    states_actions = list(zip(episodestates, episodeactions))

    T = len(episodestates)
    G = 0
    for t in range(T - 2, -1, -1):
        # retrieve current s, a, r tuple
        s = episodestates[t]
        a = episodeactions[t]
        # update G
        G = episoderewards[t+1] + GAMMA * G

        # check if we have already seen (s, a) ("first-visit")
        if (s, a) not in states_actions[:t]:
            old_q = Q[s][a]
            sample_counts[s][a] += 1
            lr = 1 / sample_counts[s][a]
            Q[s][a] = old_q + lr * (G - old_q) #Remember the incremental implementation of taking averages

            # update policy
            policy[s] = max_dict(Q[s])[0]
            state_sample_count[s]+=1

        # update delta
            biggest_change = max(biggest_change, np.abs(old_q - Q[s][a]))
        deltas.append(biggest_change)

plt.plot(deltas)
#plt.xscale('log')
plt.show()

In [ ]:
Q

In [ ]:
print_policy(policy,3,4)

In [ ]:
# find V
V = {}
for s, Qs in Q.items():
    V[s] = max_dict(Q[s])[1]

In [ ]:
V

In [ ]:
print_values(V,3,4)

In [ ]:
state_sample_count_arr = np.zeros((3, 4))
for i in range(3):
    for j in range(4):
        if (i, j) in state_sample_count:
            state_sample_count_arr[i,j] = state_sample_count[(i, j)]
    df = pd.DataFrame(state_sample_count_arr)
print(df)